<a href="https://colab.research.google.com/github/lokkster/Full-Code-Records/blob/Code/RF_parameter_tuning_for_average_gc_1110_area_normalize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
#data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new 1110 for colab.csv')
!pip install xgboost==1.6.1
import pickle
from matplotlib import pyplot

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new 1110 for colab area amplified arranged for colab.csv')


In [ ]:
from sklearn.model_selection import train_test_split
X=data.iloc[:,:5].values
Y = data.iloc[:,5:].values

train_data,test_data,train_targets,test_targets=train_test_split(X,Y,test_size=0.11,random_state=42)
print(len(train_data))
print(len(test_data))
print(test_data[0])

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

def mae(actual,pred):
     return (mean_absolute_error(actual,pred))
from sklearn.metrics import mean_absolute_percentage_error
def mape(actual,pred):
    calc=np.mean(np.absolute((actual-pred)*100/actual))
    return calc

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor()

taking default params

In [ ]:
defparam=rfr.get_params()
print(defparam)

In [ ]:
rfr.fit(train_data,train_targets)
predrfr=rfr.predict(test_data)
print(predrfr)
print(test_targets)
print("mae:",mae(test_targets,predrfr))
print("mape:",mape(test_targets,predrfr))
#xgbr.save_model('/content/drive/MyDrive/Colab Notebooks/xgbr_avggc5in.h5')

In [ ]:
from sklearn.metrics import r2_score
r2=r2_score(test_targets,predrfr,multioutput="variance_weighted")
print(r2)
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(test_targets,predrfr))
print("rmse:",rmse)

singular params


In [ ]:
rfrne=RandomForestRegressor(max_features='sqrt',max_depth=10,min_samples_split=2,min_samples_leaf=1,n_estimators=1000)
rfrne.fit(train_data,train_targets)
predrfrne=rfrne.predict(test_data)
print(predrfrne)
print(test_targets)
print("mae:",mae(test_targets,predrfrne))
print("mape:",mape(test_targets,predrfrne))

In [ ]:
#create an iterator object with write permission - model.pkl
with open("/content/drive/MyDrive/Colab Notebooks/rfr_avggc1110areanorm.pkl", 'wb') as files:
    pickle.dump(rfrne, files)

In [ ]:
from sklearn.metrics import r2_score
r2=r2_score(test_targets,predrfrne,multioutput="variance_weighted")
print(r2)
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(test_targets,predrfrne))
print("rmse:",rmse)

In [ ]:
import pickle
with open("/content/drive/MyDrive/Colab Notebooks/rfr_avggc1110areanorm.pkl" , 'rb') as f:
    new_model = pickle.load(f)

predrfrne2=new_model.predict(test_data)
print(predrfrne2)
print(test_targets)
print("mae:",mae(test_targets,predrfrne2))
print("mape:",mape(test_targets,predrfrne2))
from sklearn.metrics import r2_score
r2=r2_score(test_targets,predrfrne2,multioutput="variance_weighted")
print(r2)
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(test_targets,predrfrne))
print("rmse:",rmse)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=10, random_state=1, shuffle=True)
cvsxgbr=cross_val_score(rfrne,train_data,train_targets,cv=cv,n_jobs=-1,scoring="neg_mean_absolute_error")

print(cvsxgbr)

In [ ]:
from numpy import mean,absolute
scores=mean(absolute(cvsxgbr))

print(scores)


Randomsearchcv


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 20000, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 8,10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3,4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rfr, param_distributions = random_grid, verbose=10,n_iter = 1000, cv = 3, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_data, train_targets)

In [ ]:
print(rf_random.best_params_)

 10/29
 {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': None, 'bootstrap': True} 
 mae: 2.726143882966041
mape: 18.14193696798576

In [ ]:
rfr2=RandomForestRegressor(n_estimators=20000,min_samples_split=2,min_samples_leaf=1,max_features='auto',max_depth=10,bootstrap=True)
rfr2.fit(train_data,train_targets)
predrfr2=rfr2.predict(test_data)
print(predrfr2)
print(test_targets)
print("mae:",mae(test_targets,predrfr2))
print("mape:",mape(test_targets,predrfr2))